In [3]:
# validate the verion (3.5 version is more compatible with minio)
# jupyter-pyspark image: quay.io/jupyter/pyspark-notebook:spark-3.5.1
import pyspark 
print("PySpark:", pyspark.__version__) 
print("Spark:", spark.version) 
print("Hadoop:", spark.sparkContext._jsc.hadoopConfiguration().get("hadoop.version"))
print(spark._jsc.hadoopConfiguration().get("fs.s3a.endpoint"))

PySpark: 3.5.1
Spark: 3.5.1
Hadoop: None
http://minio.airflow:9000


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 08:10:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/11 08:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/11 08:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53592)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.p

In [4]:
data = [
    ("James", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "40288", "M", 4000),
    ("Robert", "Williams", "42114", "M", 4000),
    ("Maria", "Jones", "39192", "F", 4000),
    ("Jen", "Brown", "39193", "F", -1)
]

schema = StructType(
    [
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("id", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("salary", IntegerType(), True),
    ]
)

df = spark.createDataFrame(data, schema)

In [5]:
df.show()

+----------+---------+-----+------+------+
|first_name|last_name|   id|gender|salary|
+----------+---------+-----+------+------+
|     James|    Smith|36636|     M|  3000|
|   Michael|     Rose|40288|     M|  4000|
|    Robert| Williams|42114|     M|  4000|
|     Maria|    Jones|39192|     F|  4000|
|       Jen|    Brown|39193|     F|    -1|
+----------+---------+-----+------+------+



In [6]:
# write minio
df.write.mode("overwrite").parquet("s3a://datalake/bronze/jupyter_minio_t1")

In [ ]:
# read minio
df = spark.read.format("parquet").load("s3a://datalake/bronze/jupyter_minio_t1")
df.show()

+----------+---------+-----+------+------+
|first_name|last_name|   id|gender|salary|
+----------+---------+-----+------+------+
|    Robert| Williams|42114|     M|  4000|
|   Michael|     Rose|40288|     M|  4000|
|     James|    Smith|36636|     M|  3000|
|     Maria|    Jones|39192|     F|  4000|
|       Jen|    Brown|39193|     F|    -1|
+----------+---------+-----+------+------+



26/01/11 09:13:41 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-e02b933a-3f05-417d-86ff-2e2dc2533ea4/pyspark-6f454da7-dd94-4588-a856-4b8a206a51fd. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-e02b933a-3f05-417d-86ff-2e2dc2533ea4/pyspark-6f454da7-dd94-4588-a856-4b8a206a51fd
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun